In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd

# Extract

In [7]:
#my sample google sheet
sheet_id = '1xo1_DZEVBiVz39S5Ln3CSnagbPE78cvBDKtwqtC3hKI'
sheet_names = ['Contacts','Business','Cases','Bank_Account','Withdraw_Info']

#extract all tables from the sheet
contact_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Contacts"
df_contact = pd.read_csv(contact_url)

business_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Business"
df_business = pd.read_csv(business_url)

case_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Cases"
df_case = pd.read_csv(case_url)

bank_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Bank_Account"
df_bank = pd.read_csv(bank_url)

withdraw_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Withdraw_Info"
df_withdraw = pd.read_csv(withdraw_url)

In [8]:
df_bank

,bank_account_id,contact_id,account_type,atm,mobile_bank,active,register_branch
0,BANK001,C001,savings,yes,no,yes,Yangon
1,BANK002,C002,current,no,yes,yes,Yangon
2,BANK003,C003,fixed deposit,yes,no,yes,Yangon
3,BANK004,C004,savings,no,yes,yes,Yangon
4,BANK005,C005,current,yes,no,yes,Yangon
5,BANK006,C006,fixed deposit,no,yes,yes,Yangon
6,BANK007,C007,savings,yes,no,yes,Yangon
7,BANK008,C008,current,no,yes,yes,Yangon
8,BANK009,C009,fixed deposit,yes,no,yes,Yangon
9,BANK010,C010,savings,no,yes,yes,Yangon
